# 1. Nacimiento saludable

In [1]:
#Librerías
import pandas as pd
import os

In [49]:
os.chdir(r'C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\03. ENDES\1. Data')

# Modulo DIT
dit = pd.DataFrame()
for anio in range(2019,2023):
    df = pd.read_stata(f"dit_{anio}.dta", convert_categoricals=True)
    df.columns = df.columns.str.lower()
    dit= pd.concat([dit,df], ignore_index=True)
dit['id1'] = dit['id1'].astype(int)

# Modulo REC21
rec21 = pd.DataFrame()
for anio in range(2019,2023):
    try:
        df = pd.read_stata(f"rec21_{anio}.dta", columns = ['ID1', 'CASEID', 'BIDX', 'B4'])
    except:
        df = pd.read_stata(f"rec21_{anio}.dta", columns = ['id1', 'caseid', 'bidx', 'b4'])
    df.columns = df.columns.str.lower()
    rec21 = pd.concat([rec21,df], ignore_index=True)

# Modulo REC0111
rec0111 = pd.DataFrame()
for anio in range(2019,2023):
    try:
        df = pd.read_stata(f"rec0111_{anio}.dta", convert_categoricals=True, columns = ['ID1', 'CASEID', 'V001', 'V005', 'V012', 'V022' ,'V024' ,'V025' ,'V149' ,'V190'])
    except:
        df = pd.read_stata(f"rec0111_{anio}.dta", convert_categoricals=True, columns = ['id1','caseid', 'v001', 'v005', 'v012', 'v022' ,'v024' ,'v025' ,'v149' ,'v190'])
    df.columns = df.columns.str.lower()
    rec0111= pd.concat([rec0111,df], ignore_index=True)

# Modulo REC91
rec91 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'CASEID', 'SREGION', 'S119', 'S108N'])
    except:
        df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['id1','caseid','sregion', 's119', 's108n'])
    df.columns = df.columns.str.lower()

    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rec91 = pd.concat([rec91, df], ignore_index=True)

C:\Users\User\AppData\Local\Temp\ipykernel_34360\266190699.py:35: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'CASEID', 'SREGION', 'S119', 'S108N'])
C:\Users\User\AppData\Local\Temp\ipykernel_34360\266190699.py:37: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['id1','caseid','sregion', 's119', 's108n'])
C:\Users\User\AppData\Local\Temp\ipyke

In [64]:
# Merging datasets
mergedDIT = pd.merge(dit, rec21.loc[:,['id1','caseid', 'bidx', 'b4']], how='inner', on=['id1','caseid', 'bidx'], validate="1:1")
mergedDIT = pd.merge(mergedDIT, rec0111.loc[:,['id1','caseid', 'v001', 'v005', 'v012', 'v022' ,'v024' ,'v025' ,'v149' ,'v190']], how='inner', on=['id1','caseid'], validate="m:1")
mergedDIT = pd.merge(mergedDIT, rec91.loc[:,['id1','caseid','sregion', 's119', 's108n']], how='inner', on=['id1','caseid'], validate="m:1")

In [76]:
# Generando variables de interés
mergedDIT['v149_'] = mergedDIT['v149'].array.codes
mergedDIT['educa'] = 1  # Initialize with a default value
mergedDIT.loc[mergedDIT['v149_'] > 2, 'educa'] = 2
mergedDIT.loc[mergedDIT['v149_'] == 5, 'educa'] = 3
del mergedDIT['v149_']


In [57]:
print(mergedDIT['qi478e3'].value_counts())

qi478e3
Si                     4593
No                      608
No responde/No sabe       3
Name: count, dtype: int64


In [ ]:
# Factor de expansion
merged_final["wt"] = merged_final["v005"] / 1000000

# Generamos la variable EDADM
merged_final["edadm"] = merged_final["v008"] - merged_final["b3"]
merged_final["e_0a35"] = merged_final["edadm"] < 36
merged_final["e_0a59"] = merged_final["edadm"] < 60

# Peso al nacer
merged_final["pesonac"] = 1
merged_final.loc[(merged_final["m19"] > 2499) & (merged_final["m19"] <= 8000) & (merged_final["v012"] > 14), "pesonac"] = 2
merged_final.loc[merged_final["m19"] == 9996 & (merged_final["v012"] > 14), "pesonac"] = 3
merged_final.loc[merged_final["m19"] == 9998 & (merged_final["v012"] > 14), "pesonac"] = 4

# Labels for pesonac
labels = {1: "Menos de 2.5kg", 2: ">=2.5 kg", 3: "No pesados al nacer", 4: "No sabe, sin informacion"}
merged_final["pesonac"] = merged_final["pesonac"].map(labels)

In [ ]:
# Tabulations
tab1 = pd.crosstab(merged_final["pesonac"], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab2 = pd.crosstab([merged_final["v463a"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab3 = pd.crosstab([merged_final["v190"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab4 = pd.crosstab([merged_final["etnia"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab5 = pd.crosstab([merged_final["lmaterna"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab6 = pd.crosstab([merged_final["v025"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab7 = pd.crosstab([merged_final["sregion"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")

# Tamaño de la niña o niño al nacer
merged_final["ch_size_birth"] = pd.cut(merged_final["m18"], bins=[-1, 2, 4, 5, 8, 9], labels=["Promedio o mayor", "Menor que promedio", "Muy pequeño", "No sabe/sin información"], right=False)
tab8 = pd.crosstab(merged_final["ch_size_birth"], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab9 = pd.crosstab([merged_final["v463a"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab10 = pd.crosstab([merged_final["v190"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab11 = pd.crosstab([merged_final["etnia"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab12 = pd.crosstab([merged_final["lmaterna"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab13 = pd.crosstab([merged_final["v025"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab14 = pd.crosstab([merged_final["sregion"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
